In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List
import io
import pydotplus
from IPython.display import Image

RANDOM_SEED = 0xdeadbeef

## Introduction

In this tutorial, we develop a basic implementation of a decision tree from scratch. 

Decision Trees are a relatively simple and interpretable type of model that can be used both for classification and regression. In this lab we will look at how they are implemented for **classification**.

In their simplest form, a Decision Tree is a *Binary Tree* that splits the data
at each node into two parts, based on some criterion.

As always, we start out by creating some synthetic data. You should notice that this data is not linearly separable, meaning there is no line such that all red points are on one side and all the yellow points on the other.

One nice feature of Decision Trees is that they can handle situations like this without any "tricks".

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(
    n_samples=800,
    n_features=2,
    centers=np.array([
        [-1., -1.],
        [1., -1],
        [1., 1.],
        [-1., 1.],
    ]),
    cluster_std=0.25,
    random_state=RANDOM_SEED,
)
y[y == 2] = 0
y[y == 3] = 1

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap="autumn", edgecolors='b')

### Gini Impurity

In class you have seen the *Gini Impurity* as one possible criterion to split data points.

The Gini Impurity is computed on a collection of class labels. Assume we are given a list of $n$ class labels. Each label can have one of $k$ discrete values. Then the Gini Impurity is computed as: $\sum_{c = 1}^{k} p_c(1 - p_c)$. In this formula $p_c = \frac{n_c}{n}$ is the fraction of labels of class $c$.

In class you have seen a simplified version with only 2 classes: "Yes" (1) and "No" (0). In that case the Gini Impurity simplifies to: $1 - p_1^2 - p_0^2$.

In the next cell, we implement the Gini Impurity for a given list of labels. For simplicity, we assume that each label can only have a value of either 0 or 1.

In [ ]:
def gini_impurity(labels: np.array) -> float:
  """
  Compute Gini Impurity for labels

  labels: a 1D array of labels with values of either 0 or 1

  return: the Gini Impurity
  """
  n = len(labels)
  p0 = (labels == 0).sum() / n
  p1 = (labels == 1).sum() / n
  return 1. - p0*p0 - p1*p1

When deciding where to split, we compute the a weighted average of the impurities for the left and right sub-trees: $p_{left} \times G_{left} + p_{right} \times G_{right}$.

Here $G_{left}$ is the Gini Impurity of the left sub-tree and $p_{left}$ is the fraction of data assigned to the left sub-tree (and analogous for $G_{right}$ and $p_{right}$).

In [ ]:
def split_impurity(labels_left: np.array, labels_right: np.array) -> float:
  """
  Computed the weighted average Gini Impurity for a given split of labels.

  left_labels: 1D array of labels with values of either 0 or 1 that belong to the left sub-tree
  right_labels: 1D array of labels with values of either 0 or 1 that belong to the right sub-tree

  return: weighted average of the Gini Impurities of the left and right sub-trees
  """
  n_left = len(labels_left)
  n_right = len(labels_right)
  p_left = n_left / (n_left + n_right)
  p_right = n_right / (n_left + n_right)
  gini_left = gini_impurity(labels_left)
  gini_right = gini_impurity(labels_right)
  return p_left*gini_left + p_right*gini_right

The function `split_impurity` that you just implemented can be used to decide between multiple potential splits by selecting with the lowest value.

To find the best split for numerical data you would do the following (see also the lecture slides, as a reminder):
* 1.  sort numeric data from lowest to highest
* 2.  compute average of every pair of adjacent values as candidate split threshold
* 3.  compute `split_impurity` for each possible split
* 4.  select the one with the minimum `split_impurity`


In the next cell this procedure is implemented to find the best location to split numeric data.

In [ ]:
def split_location(x: np.array, labels: np.array) -> float:
  """
  Find the best split location for given numeric data

  x: a 1D array of numeric (float) values 
  labels: a 1D array of labels (0 or 1) associated with x
  
  return: the threshold t where to split the data, such that data points <= t go to the left sub-tree and data points > t go to the right sub-tree
  """
  sorted = np.sort(x)
  threshs = (sorted[:-1] + sorted[1:]) / 2.

  best_thresh = -1.
  best_score = np.inf
  for th in threshs:
    left = labels[x <= th]
    right = labels[x > th]
    score = split_impurity(left, right)

    if score < best_score:
      best_thresh = th
      best_score = score
  
  return best_thresh

### Decision Tree Implementation

We now have everything we need to implement our own Decision Tree. In the next 2 cells we have the code for one possible implementation.
In the next cell we have the class `DecisionTreeNode` representing a node inside a Decision Tree. 


In [ ]:
class DecisionTreeNode:

  def __init__(
    self,
    data: np.array,
    labels: np.array,
    split_dimension: int,
    min_points_per_leaf: int,
  ):
    # a 2D array of numerical data, rows (dim 0) are samples and columns (dim 1) are features
    self.data = data
    # a 1D array of labels (0 or 1) associated with data
    self.labels = labels
    # each node can only split on 1 feature dimension
    self.split_dimension = split_dimension
    # we won't split nodes that have too few elements
    self.min_points_per_leaf = min_points_per_leaf

    # left sub-tree (if not a leaf)
    self.left: Optional[DecisionTreeNode] = None
    # right sub-tree (if not a leaf)
    self.right: Optional[DecisionTreeNode] = None
    # the decision threshold for this node (if not a leaf)
    self.split_threshold: Optional[float] = None
    # if this node is a leaf, we will store its label here
    self.leaf_label: Optional[int] = None 

    # check whether a nude is pure, meaning there are only samples from 1 class
    # in that case we won't have split any further
    pure_node = len(set(self.labels)) == 1

    # we won't split nodes with very few elements, this is a type of pruning
    # you've seen in the lecture, do you remember which one?
    too_small = len(self.labels) <= self.min_points_per_leaf

    if not pure_node and not too_small:
      # we separated out the splitting logic
      self.split()
    else:
      # for leaf nodes, we store the label, which is just the majority of 
      # the data labels
      self.leaf_label = np.argmax(np.bincount(self.labels))

  def split(self):
    
    # we use the function implemented earlier to find the optimal splitting
    # threshold.
    # we use self.split_dimension as the dimension to split along
    self.split_threshold = split_location(
        x=self.data[:, self.split_dimension],
        labels=self.labels,
    )

    # compute row indices of which data points will go to the left and right sub-trees
    left_ixs = (self.data[:, self.split_dimension] <= self.split_threshold)
    right_ixs = (self.data[:, self.split_dimension] > self.split_threshold)

    # each child node will use a different column to split. we use modulo % to wrap around.
    next_split = (self.split_dimension + 1) % self.data.shape[1]

    # initialize left / right children
    # we only pass the data and labels selected by the indices that we computed earlier
    self.left = DecisionTreeNode(
        data=self.data[left_ixs, :],
        labels=self.labels[left_ixs],
        split_dimension=next_split,
        min_points_per_leaf=self.min_points_per_leaf,
    )
    self.right = DecisionTreeNode(
        data=self.data[right_ixs, :],
        labels=self.labels[right_ixs],
        split_dimension=next_split,
        min_points_per_leaf=self.min_points_per_leaf,
    )

  def predict(self, X) -> np.array:
    # predict labels for new, unseen data X

    n_samples = X.shape[0]

    if self.leaf_label is not None:
      # if current node is a leaf, we just return its label (one for each row in the input data)
      return np.ones(n_samples, dtype=int) * self.leaf_label
    else:
      # for non-leaf nodes, we have to split the data and get predictions from the child nodes

      # initialize result variable
      result = np.zeros(n_samples, dtype=int)

      # compute row indices of which data points belong to which child sub-tree
      left_ixs = (X[:, self.split_dimension] <= self.split_threshold)
      right_ixs = (X[:, self.split_dimension] > self.split_threshold)

      # get predictions from sub-trees
      result[left_ixs] = self.left.predict(X[left_ixs, :])
      result[right_ixs] = self.right.predict(X[right_ixs, :])
      
      return result
    
  def print(self, prefix: str = ""):
    # Helper method to 'pretty' print the Decision Tree
    # You do not have to understand this.
    if self.leaf_label is not None:
      print(f"{prefix} Predict {self.leaf_label}")
    else:
      print(f"{prefix} Dimension {self.split_dimension} <= {self.split_threshold:.3f}")
      self.left.print(prefix=prefix + "\t")
      print(f"{prefix} Dimension {self.split_dimension} > {self.split_threshold:.3f}")
      self.right.print(prefix=prefix + "\t")


In the cell below, we provide a light wrapper class `CustomDecisionTree` that exposes the usual `sklearn` interface with a `.fit` and `.predict` method.

In [ ]:
class CustomDecisionTree:

  def __init__(
    self,
    min_points_per_leaf: int = 5,
  ):
    self.root: Optional[DecisionTreeNode] = None
    self.min_points_per_leaf = min_points_per_leaf
  
  def fit(self, X, y):
    self.root = DecisionTreeNode(
        data=X,
        labels=y,
        split_dimension=0,
        min_points_per_leaf=self.min_points_per_leaf,
    )
    return self

  def predict(self, X):
    if self.root is None:
      raise ValueError(f"you have to call `fit` before you can get predictions")
    return self.root.predict(X)

  def print(self):
    print("ROOT:")
    if self.root is not None:
      self.root.print(prefix="\t")
    else:
      print(f"\tnot yet fitted")


* Question: Does our implementation apply any pruning technique? If so, which kind of pruning?*


### Trying it out

Below, we apply our `CustomDecisionTree` to the initial data `X` and `y`. We will also visualize its decision boundaries.


*Note*: we provide the function `plot_decision_boundaries` in the next cell, which we will use to visualize the decision regions of different classifiers.

In [ ]:
# Helper function to visualize the decision boundaries and regions of a classifier
# You do not need to read it or understand how it works

def plot_decision_boundaries(
  classifier,
  data: np.array,
  labels: np.array,
):
  """
  Plot the decision boundaries and regions for a classifier on 2D data

  classifier: any classifier implementing a .predict method that returns integer labels
  data: a 2D numpy array of shape [n_samples, 2], containing data points to be classified
  labels: a 1D numpy array of shape [n_samples], containing the labels for data 
  """
  x_lo = np.min(data[:, 0]) - .5
  x_hi = np.max(data[:, 0]) + .5
  y_lo = np.min(data[:, 1]) - .5
  y_hi = np.max(data[:, 1]) + .5

  xx, yy = np.meshgrid(
      np.arange(x_lo, x_hi, 0.02),
      np.arange(y_lo, y_hi, 0.02),
  )

  grid_preds = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
  grid_preds = grid_preds.reshape(xx.shape)

  plt.contourf(xx, yy, grid_preds, alpha=.6, cmap='autumn')
  plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='autumn', edgecolors='b')

  plt.show()


Have a look at the print-out of the decision tree below and the plot of it decisions on our `X` and `y`.

In [ ]:
custom_tree = CustomDecisionTree()
custom_tree.fit(X, y)
custom_tree.print()
plot_decision_boundaries(custom_tree, X, y)